In [ ]:
# 步骤 2：导入必要的库
import os
import openai
import numpy as np
import faiss
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import logging
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [ ]:
# 步骤 3：配置API密钥和基础URL
# 请确保已经在系统环境变量中设置了 OPENAI_API_KEY 和 OPENAI_API_BASE

openai.api_key = os.getenv("OPENAI_API_KEY", "sk-16a90ba86cfc4dcf9402bea1309c9021")
openai.api_base = os.getenv("OPENAI_API_BASE", "https://api.deepseek.com")

# 验证API密钥是否设置
if not openai.api_key:
    logging.warning("OPENAI_API_KEY未设置。请在环境变量中设置它以启用API调用。")


In [ ]:
# 步骤 4：加载嵌入模型

# 加载嵌入模型
device = torch.device("cpu")  # 强制使用CPU
try:
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2').to(device)
    logging.info("嵌入模型加载成功！")
except Exception as e:
    logging.error(f"加载嵌入模型时发生错误: {e}")


In [ ]:
# 步骤 5：定义嵌入函数

# 嵌入函数（批量处理）
def embed_texts(texts, batch_size=16):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        # 使用CLS token的输出作为句子的嵌入
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)


In [ ]:
# 步骤 6：加载文档数据

# 加载文档数据（CSV文件）
def load_documents(file_path, nrows=100):
    try:
        df = pd.read_csv(file_path, nrows=nrows)
        df = df.dropna(subset=['text'])
        logging.info(f"成功加载了 {len(df)} 条文档。")
        return df
    except FileNotFoundError:
        logging.error(f"文件 {file_path} 未找到。请检查路径是否正确。")
        return pd.DataFrame(columns=['text'])
    except Exception as e:
        logging.error(f"加载文档时发生错误: {e}")
        return pd.DataFrame(columns=['text'])


In [ ]:
# 步骤 7：构建FAISS索引

# 构建FAISS索引
def build_faiss_index(embeddings, use_quantization=False):
    dimension = embeddings.shape[1]
    
    if use_quantization:
        # 使用Product Quantization进行压缩
        nlist = 100  # 聚类数
        quantizer = faiss.IndexFlatL2(dimension)
        index = faiss.IndexIVFPQ(quantizer, dimension, nlist, 16, 8)  # 16 bytes per vector, 8 subquantizers
        index.train(embeddings)
        index.add(embeddings)
        logging.info("使用量化的FAISS索引已构建。")
    else:
        # 使用简单的扁平索引（不量化）
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings)
        logging.info("使用扁平FAISS索引已构建。")
    
    return index


In [ ]:
# 步骤 8：检索相关文档

# 检索相关文档
def retrieve_relevant_documents(index, query_embedding, texts, top_k=2):
    distances, indices = index.search(np.array([query_embedding]), top_k)
    return [texts[i] for i in indices[0]]


In [ ]:
# 步骤 9：生成回答

# 使用DeepSeek生成回答
def generate_response(prompt):
    if not openai.api_key:
        return "API密钥未设置。请设置OPENAI_API_KEY环境变量。"
    
    try:
        response = openai.ChatCompletion.create(
            model="deepseek-chat",
            messages=[
                {"role": "user", "content": prompt},
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"API调用失败: {e}"


In [ ]:
# 步骤 10：主流程

# 主流程
def main(use_csv=True, file_path="documents.csv", nrows=100, use_quantization=False, top_k=2):
    try:
        if use_csv:
            # 使用CSV加载文档
            logging.info("正在加载CSV文档...")
            df = load_documents(file_path, nrows=nrows)
            
            if df.empty:
                logging.error("未加载到任何文档。请检查CSV文件。")
                return
            
            texts = df['text'].tolist()
        else:
            # 使用默认文档列表
            logging.info("未使用CSV，使用默认文档。")
            texts = [
                "这是第一段默认的文本。",
                "这是第二段默认的文本。",
                "这是第三段默认的文本。"
            ]
            
            if not texts:
                logging.error("默认文档列表为空。")
                return
        
        # 生成嵌入
        logging.info("正在生成嵌入...")
        embeddings = embed_texts(texts)
        
        # 构建FAISS索引
        logging.info("正在构建FAISS索引...")
        faiss_index = build_faiss_index(embeddings, use_quantization=use_quantization)
        
        # 获取用户输入（这里使用固定的问题）
        user_input = "CHIMA是谁?"
        
        # 生成查询嵌入
        logging.info("正在生成查询嵌入...")
        query_embedding = embed_texts([user_input])[0]
        
        # 检索相关文档
        logging.info("正在检索相关文档...")
        relevant_docs = retrieve_relevant_documents(faiss_index, query_embedding, texts, top_k=top_k)
        
        if not relevant_docs:
            logging.warning("未检索到相关文档。")
            return
        
        # 将检索到的文档作为上下文
        context = "\n".join(relevant_docs)
        logging.info(f"检索到的上下文内容如下：\n{context}")
        
        # 使用DeepSeek生成回答
        logging.info("正在生成AI回答...")
        #prompt = f"根据以下上下文回答用户问题：\n\n上下文：\n{context}\n\n问题：\n{user_input}"
        prompt = f"根据以下上下文回答用户问题：\n\n上下文：\n{context}\n\n问题：\n{user_input}。如果缺少上下文则根据你的知识回答。"
        ai_response = generate_response(prompt)
        
        print("\nAI回答：")
        print(ai_response)
    
    except Exception as e:
        logging.error(f"发生错误: {e}")


In [ ]:
# 步骤 11：运行主流程

# 运行主流程，使用CSV加载文档
main(use_csv=True, file_path="documents.csv", nrows=100, use_quantization=False, top_k=2)
#main(use_csv=False)
